## A gentle 10-minute introduction to Ray AI Runitime (Ray AIR)

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>

As part of Ray 2.0, Ray AI Runtime (AIR) is an open-source toolkit for building end-to-end simple and scalable ML applications. 

Ray AI Runtime focuses on two functional aspects:
 * It provides scalability by leveraging Ray’s distributed compute layer for ML workloads.
 * It is designed to interoperate with other systems for storage and metadata needs.

Ray AIR consists of five key components:

 * Data processing ([Ray Data](https://docs.ray.io/en/latest/data/dataset.html))
 * Model Training ([Ray Train](https://docs.ray.io/en/latest/train/train.html))
 * Hyperparameter Tuning ([Ray Tune](https://docs.ray.io/en/latest/tune/index.html))
 * Model Serving ([Ray Serve](https://docs.ray.io/en/latest/serve/index.html)).
 * Reinforcement Learning ([Ray RLlib](https://docs.ray.io/en/latest/rllib/index.html))
 
 <img src = "images/ray-air.svg" width="60%" height="30%">
 
📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>
 
### Learning objectives:
  * How to use Ray AIR as a unified toolkit to write an end-to-end ML application in a single Python script
  * Use out-of-box Preprocessors
  * Load model from the best model checkpoint and use for batch inference
  * Deploy best checkpoint model and use for online inference

In [8]:
import logging, os, random, warnings
import ray
import pandas as pd

In [9]:
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [10]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.0.0rc0
Dashboard:,http://127.0.0.1:8266


### Create Ray data from an S3 CSV datasource

In [11]:
dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")

# Split data into train and validation.
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)
test_dataset = valid_dataset.drop_columns(["target"])

Map_Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.06it/s]


### Use out-of-the-box Preprocessors
This preprocessor is automatically used in the training function to `fit` and `tranform` your datasets for training and validation. You don't have to explicitly call the preprocess before training or inference. Ray AIR toolkit automatically does that for you. 

We are going to scaler a few features like `mean radius` and `mean texture`.

In [12]:
from ray.data.preprocessors import StandardScaler

# Create a preprocessor to scale some columns
columns_to_scale = ["mean radius", "mean texture"]
preprocessor = StandardScaler(columns=columns_to_scale)

### Use AIR Trainers for supported ML frameworks
Use the Ray AIR trainer `XGBoostTrainer` with simple steps:
 1. define the parallelism for Ray compute
 2. define the XGBoost parameters for training
 3. supply the preprocessor for fitting and transforming dataset during training and validation
 4. provide the datasets for training and validation
 5. invoke `trainer.fit()` 
 
 Simple API that does a lot behind the scenes for you!

In [13]:
from ray.air.config import ScalingConfig
from ray.train.xgboost import XGBoostTrainer

trainer = XGBoostTrainer(
    scaling_config=ScalingConfig(
        # Number of workers to use for data parallelism.
        num_workers=2,
        # Whether to use GPU acceleration.
        use_gpu=False),
    label_column="target",
    num_boost_round=20,
    params={
        # XGBoost specific params
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    # our train and validation dataset and preprocessor
    datasets={"train": train_dataset, "valid": valid_dataset},
    preprocessor=preprocessor,
)

##### Fit the trainer

In [14]:
result = trainer.fit()
# print(result.metrics)

Trial name,status,loc,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_e50dc_00000,TERMINATED,127.0.0.1:47857,21,4.72733,0.0184957,0,0.0893879


(_map_block_nosplit pid=47367) E0808 11:14:08.737874000 6140850176 chttp2_transport.cc:1111]          Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"
(XGBoostTrainer pid=47857) 2022-08-08 11:14:11,865	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(_RemoteRayXGBoostActor pid=47871) 2022-08-08 11:14:13,036	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.get_resource_ids` is a private attribute and access will be removed in a future Ray version.
(_RemoteRayXGBoostActor pid=47871)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/workers/default_worker.py", line 237, in <module>
(_RemoteRayXGBoostActor pid=47871)     ray._private.worker.global_worker.main_loop()
(_RemoteRayXGBoostActor pid=47871)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/worker.py", line 754, in main_loop
(_Remo

Result for XGBoostTrainer_e50dc_00000:
  date: 2022-08-08_11-14-15
  done: false
  experiment_id: 21061788bbe0491aaa2a4cc8ee1c4a65
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 47857
  time_since_restore: 4.161697149276733
  time_this_iter_s: 4.161697149276733
  time_total_s: 4.161697149276733
  timestamp: 1659982455
  timesteps_since_restore: 0
  train-error: 0.02261306532663317
  train-logloss: 0.464117960489575
  training_iteration: 1
  trial_id: e50dc_00000
  valid-error: 0.11695906432748537
  valid-logloss: 0.5025240946234318
  warmup_time: 0.0028510093688964844
  


(XGBoostTrainer pid=47857) 2022-08-08 11:14:15,198	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 3.35 seconds (1.77 pure XGBoost training time).


Result for XGBoostTrainer_e50dc_00000:
  date: 2022-08-08_11-14-15
  done: true
  experiment_id: 21061788bbe0491aaa2a4cc8ee1c4a65
  experiment_tag: '0'
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 47857
  time_since_restore: 4.727331876754761
  time_this_iter_s: 0.4802539348602295
  time_total_s: 4.727331876754761
  timestamp: 1659982455
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01849572773292735
  training_iteration: 21
  trial_id: e50dc_00000
  valid-error: 0.04093567251461988
  valid-logloss: 0.08938791319913073
  warmup_time: 0.0028510093688964844
  


### Use AIR Tuner for hyperparameter search

What if you want to do hyperparameter optimization during training and use the best config for the model? Well, you can then use Tuner and supply your training function, Trainer, as part of the argument, along 
with other Tuner configuration. 

Again, simple steps:
 1. define your hyperparameter space
 2. define `TuneConfig` for number of trials and parallelism 
 3. invoke `tuner.fit()`

In [15]:
from ray import tune

param_space = {"params": {"max_depth": tune.randint(1, 9)}}
metric = "train-logloss"
our_mode="min"

In [16]:
from ray.tune.tuner import Tuner, TuneConfig
from ray.air.config import RunConfig

tuner = Tuner(
    trainer,
    param_space=param_space,
    tune_config=TuneConfig(num_samples=5, metric=metric, mode=our_mode),
)
# Execute tuning.
result_grid = tuner.fit()

Trial name,status,loc,params/max_depth,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_20b15_00000,TERMINATED,127.0.0.1:48000,8,21,3.84942,0.0184957,0,0.0893879
XGBoostTrainer_20b15_00001,TERMINATED,127.0.0.1:48007,7,21,4.99472,0.0184957,0,0.0893879
XGBoostTrainer_20b15_00002,TERMINATED,127.0.0.1:48008,3,21,4.9968,0.0215151,0,0.0765915
XGBoostTrainer_20b15_00003,TERMINATED,127.0.0.1:48088,2,21,4.79494,0.0405455,0.00502513,0.0916641
XGBoostTrainer_20b15_00004,TERMINATED,127.0.0.1:48105,1,21,3.06223,0.0955215,0.0175879,0.112007


(XGBoostTrainer pid=48000) 2022-08-08 11:15:49,572	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(_RemoteRayXGBoostActor pid=48013) 2022-08-08 11:15:50,749	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.get_resource_ids` is a private attribute and access will be removed in a future Ray version.
(_RemoteRayXGBoostActor pid=48013)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/workers/default_worker.py", line 237, in <module>
(_RemoteRayXGBoostActor pid=48013)     ray._private.worker.global_worker.main_loop()
(_RemoteRayXGBoostActor pid=48013)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/worker.py", line 754, in main_loop
(_RemoteRayXGBoostActor pid=48013)     self.core_worker.run_task_loop()
(_RemoteRayXGBoostActor pid=48013)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/func

Result for XGBoostTrainer_20b15_00000:
  date: 2022-08-08_11-15-52
  done: false
  experiment_id: efc7e92ab7a94bb1a5092af520be71e2
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 48000
  time_since_restore: 3.4313011169433594
  time_this_iter_s: 3.4313011169433594
  time_total_s: 3.4313011169433594
  timestamp: 1659982552
  timesteps_since_restore: 0
  train-error: 0.02261306532663317
  train-logloss: 0.464117960489575
  training_iteration: 1
  trial_id: 20b15_00000
  valid-error: 0.11695906432748537
  valid-logloss: 0.5025240946234318
  warmup_time: 0.002680063247680664
  


(XGBoostTrainer pid=48000) 2022-08-08 11:15:52,999	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 3.44 seconds (2.08 pure XGBoost training time).


(scheduler +1h43m11s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +1h43m11s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
Result for XGBoostTrainer_20b15_00000:
  date: 2022-08-08_11-15-53
  done: true
  experiment_id: efc7e92ab7a94bb1a5092af520be71e2
  experiment_tag: 0_max_depth=8
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 48000
  time_since_restore: 3.8494162559509277
  time_this_iter_s: 0.3568880558013916
  time_total_s: 3.8494162559509277
  timestamp: 1659982553
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01849572773292735
  training_iteration: 21
  trial_id: 20b15_00000
  valid-error: 0.04093567251461988
  valid-logloss: 0.08938791319913073
  warmup_time

...

(XGBoostTrainer pid=48088) 2022-08-08 11:15:56,771	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(_RemoteRayXGBoostActor pid=48113) 2022-08-08 11:15:57,983	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.get_resource_ids` is a private attribute and access will be removed in a future Ray version.
(_RemoteRayXGBoostActor pid=48113)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/workers/default_worker.py", line 237, in <module>
(_RemoteRayXGBoostActor pid=48113)     ray._private.worker.global_worker.main_loop()
(_RemoteRayXGBoostActor pid=48113)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/worker.py", line 754, in main_loop
(_RemoteRayXGBoostActor pid=48113)     self.core_worker.run_task_loop()
(_RemoteRayXGBoostActor pid=48113)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/func

Result for XGBoostTrainer_20b15_00003:
  date: 2022-08-08_11-16-00
  done: false
  experiment_id: 5226d727ea2a404a88093fd5659a2bd6
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 48088
  time_since_restore: 4.29737401008606
  time_this_iter_s: 4.29737401008606
  time_total_s: 4.29737401008606
  timestamp: 1659982560
  timesteps_since_restore: 0
  train-error: 0.04773869346733668
  train-logloss: 0.4862994935794092
  training_iteration: 1
  trial_id: 20b15_00003
  valid-error: 0.09941520467836257
  valid-logloss: 0.5120853461020174
  warmup_time: 0.0025920867919921875
  


(XGBoostTrainer pid=48088) 2022-08-08 11:16:00,156	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 3.40 seconds (1.95 pure XGBoost training time).


Result for XGBoostTrainer_20b15_00003:
  date: 2022-08-08_11-16-00
  done: true
  experiment_id: 5226d727ea2a404a88093fd5659a2bd6
  experiment_tag: 3_max_depth=2
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 48088
  time_since_restore: 4.794938802719116
  time_this_iter_s: 0.4245119094848633
  time_total_s: 4.794938802719116
  timestamp: 1659982560
  timesteps_since_restore: 0
  train-error: 0.0050251256281407
  train-logloss: 0.04054545671047278
  training_iteration: 21
  trial_id: 20b15_00003
  valid-error: 0.02923976608187134
  valid-logloss: 0.09166410522894901
  warmup_time: 0.0025920867919921875
  
Result for XGBoostTrainer_20b15_00004:
  date: 2022-08-08_11-16-00
  done: false
  experiment_id: 24833585e99f4cfdba6bbf33c2be25bc
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 48105
  time_since_restore: 2.7731900215148926
  time_this_iter_s: 2.7731900215148926
  time_total_s: 2.77319002151489

(XGBoostTrainer pid=48105) 2022-08-08 11:16:00,834	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 2.68 seconds (1.18 pure XGBoost training time).


In [18]:
# Fetch the best result with its best hyperparameter config 
best_result = result_grid.get_best_result()
print("Best Result:", best_result)

Best Result: Result(metrics={'train-logloss': 0.01849572773292735, 'train-error': 0.0, 'valid-logloss': 0.08938791319913073, 'valid-error': 0.04093567251461988, 'done': True, 'trial_id': '20b15_00000', 'experiment_tag': '0_max_depth=8'}, error=None, log_dir=PosixPath('/Users/jules/ray_results/XGBoostTrainer_2022-08-08_11-15-48/XGBoostTrainer_20b15_00000_0_max_depth=8_2022-08-08_11-15-48'))


### Ray AIR Checkpoints

The AIR trainers, tuners, and custom pretrained model generate Checkpoints. An AIR Checkpoint is a format for models that are used across different components of the Ray AI Runtime. This common format allows easy interoperability among AIR components and seamless integration with external supported machine learning frameworks. Read more
about [Checkpoints]().

<img src="images/checkpoints.jpeg" height="25%" and width="50%"> 

### Use AIR `BatchPreditor` for batch prediction
Once you have trained and tuned your model, create a batch predictor from best model using the `best_result.checkpoint` and do batch inference. 

In [19]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.xgboost import XGBoostPredictor

batch_predictor = BatchPredictor.from_checkpoint(best_result.checkpoint, XGBoostPredictor)

predicted_probabilities = batch_predictor.predict(test_dataset)
print("PREDICTED PROBABILITIES")
predicted_probabilities.show()

Map Progress (1 actors 0 pending): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.31it/s]

PREDICTED PROBABILITIES
{'predictions': 0.9964648485183716}
{'predictions': 0.9951295852661133}
{'predictions': 0.0037899704184383154}
{'predictions': 0.9964648485183716}
{'predictions': 0.9969868063926697}
{'predictions': 0.9947494864463806}
{'predictions': 0.9899886250495911}
{'predictions': 0.9952162504196167}
{'predictions': 0.3375702202320099}
{'predictions': 0.9766711592674255}
{'predictions': 0.0037899704184383154}
{'predictions': 0.9948934316635132}
{'predictions': 0.9472665786743164}
{'predictions': 0.989780068397522}
{'predictions': 0.9952002763748169}
{'predictions': 0.18953870236873627}
{'predictions': 0.2149435132741928}
{'predictions': 0.99428790807724}
{'predictions': 0.9890844225883484}
{'predictions': 0.0037899704184383154}


### Use `PredictorDeployment` for online inference

Deploy the best model as an inference service by using Ray Serve and the `PredictorDeployment` class.

In [21]:
from ray import serve
from fastapi import Request
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import json_request


async def adapter(request: Request):
    content = await request.json()
    print(content)
    return pd.DataFrame.from_dict(content)


serve.start(detached=True)
deployment = PredictorDeployment.options(name="XGBoostService", num_replicas=2, route_prefix="/rayair")

deployment.deploy(
    XGBoostPredictor, best_result.checkpoint, batching_params=False, http_adapter=adapter
)

print(deployment.url)

(ServeController pid=48735) INFO 2022-08-08 11:22:06,408 controller 48735 deployment_state.py:1189 - Stopping 1 replicas of deployment 'XGBoostService' with outdated versions.
(ServeController pid=48735) INFO 2022-08-08 11:22:08,538 controller 48735 deployment_state.py:1232 - Adding 1 replicas to deployment 'XGBoostService'.
(ServeController pid=48735) INFO 2022-08-08 11:22:09,481 controller 48735 deployment_state.py:1189 - Stopping 1 replicas of deployment 'XGBoostService' with outdated versions.
(ServeController pid=48735) INFO 2022-08-08 11:22:11,637 controller 48735 deployment_state.py:1232 - Adding 1 replicas to deployment 'XGBoostService'.


http://127.0.0.1:8000/rayair


After deploying the service, you can send requests to it.

In [22]:
import requests

sample_input = test_dataset.take(1)
sample_input = dict(sample_input[0])

output = requests.post(deployment.url, json=[sample_input]).json()
print(output)

[{'predictions': 0.9964648485183716}]
(ServeReplica:XGBoostService pid=48778) [{'mean radius': 11.06, 'mean texture': 14.83, 'mean perimeter': 70.31, 'mean area': 378.2, 'mean smoothness': 0.07741, 'mean compactness': 0.04768, 'mean concavity': 0.02712, 'mean concave points': 0.007246, 'mean symmetry': 0.1535, 'mean fractal dimension': 0.06214, 'radius error': 0.1855, 'texture error': 0.6881, 'perimeter error': 1.263, 'area error': 12.98, 'smoothness error': 0.004259, 'compactness error': 0.01469, 'concavity error': 0.0194, 'concave points error': 0.004168, 'symmetry error': 0.01191, 'fractal dimension error': 0.003537, 'worst radius': 12.68, 'worst texture': 20.35, 'worst perimeter': 80.79, 'worst area': 496.7, 'worst smoothness': 0.112, 'worst compactness': 0.1879, 'worst concavity': 0.2079, 'worst concave points': 0.05556, 'worst symmetry': 0.259, 'worst fractal dimension': 0.09158}]


(HTTPProxyActor pid=48737) INFO 2022-08-08 11:22:30,238 http_proxy 127.0.0.1 http_proxy.py:315 - POST /rayair 307 3.4ms
(HTTPProxyActor pid=48737) INFO 2022-08-08 11:22:30,250 http_proxy 127.0.0.1 http_proxy.py:315 - POST /rayair 200 10.8ms
(ServeReplica:XGBoostService pid=48775) INFO 2022-08-08 11:22:30,237 XGBoostService XGBoostService#QGWmsh replica.py:482 - HANDLE __call__ OK 0.2ms
(ServeReplica:XGBoostService pid=48778) INFO 2022-08-08 11:22:30,249 XGBoostService XGBoostService#hgpRar replica.py:482 - HANDLE __call__ OK 7.8ms


In [23]:
ray.shutdown()

### Homework

1. Have a go at Ray AIR examples in the documentation.

 📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>

Done! 🍻
 